In [1]:
import numpy as np
from sklearn import svm
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.neighbors import NearestNeighbors
import random

In [2]:
dataset = np.loadtxt('m_train_ori.csv',delimiter=',')
testset = np.loadtxt('m_test_ori.csv',delimiter=',')

dataset_all = dataset[0:dataset.shape[0],0:dataset.shape[1]-2]
dataset_all = preprocessing.scale(dataset_all)
dataset_all_label = dataset[0:dataset.shape[0],dataset.shape[1]-2]

In [3]:
dataset1 = []
dataset2 = []
dataset3 = []
dataset4 = []
dataset5 = []

for r in range(dataset.shape[0]):
    if dataset[r,dataset.shape[1]-2] == 1:
        dataset1.append(dataset[r])
    elif dataset[r,dataset.shape[1]-2] == 2:
        dataset2.append(dataset[r])
    elif dataset[r,dataset.shape[1]-2] == 3:
        dataset3.append(dataset[r])
    elif dataset[r,dataset.shape[1]-2] == 4:
        dataset4.append(dataset[r])
    elif dataset[r,dataset.shape[1]-2] == 5:
        dataset5.append(dataset[r])
        
dataset1 = np.array(dataset1)
dataset2 = np.array(dataset2)
dataset3 = np.array(dataset3)
dataset4 = np.array(dataset4)
dataset5 = np.array(dataset5)

datasets = [dataset1, dataset2, dataset3, dataset4, dataset5]

sample_num = max([dataset1.shape[0], dataset2.shape[0], dataset3.shape[0], dataset4.shape[0], dataset5.shape[0]])

for k in range(len(datasets)):
    if datasets[k].shape[0] != sample_num:
        cur_num = datasets[k].shape[0]
        sample_rate = int(sample_num / cur_num)
        neigh = NearestNeighbors(n_neighbors = 2)
        neigh.fit(datasets[k])
        for i in range(cur_num):
            neis = neigh.kneighbors([datasets[k][i]])
            for j in range(sample_rate):
                l = random.randrange(2)
                new_row = datasets[k][i] + random.random() * (datasets[k][neis[1][0][l]] - datasets[k][i])
                datasets[k] = np.vstack((datasets[k], new_row))
                
#print(datasets[4].shape)

test_row = testset.shape[0]
test_col = testset.shape[1]

test = testset[0:test_row,0:test_col-2]
test = preprocessing.scale(test)

test_label_1 = testset[0:test_row,test_col-2]

labels = []
labels_train = []
judge = 1
judge_train = 1

for i in range(len(datasets) - 1):
    for j in range(i + 1,len(datasets)):
        dataset = np.vstack((datasets[i],datasets[j]))

        row = dataset.shape[0]
        col = dataset.shape[1]
        
        pca = PCA(n_components = 5)
        train = dataset[0:row,0:col-2]
        train = preprocessing.scale(train)
        train = pca.fit_transform(train)
        
        train_label_1 = dataset[0:row,col-2]
        
        clf = svm.SVC()
        clf.fit(train, train_label_1)
        test_tmp = pca.transform(test)
        dataset_temp = pca.transform(dataset_all)
        
        if judge_train == 1:
            labels_train = np.array([clf.predict(dataset_temp)]).T
            judge_train = 0
        else:
            labels_train = np.hstack((labels_train,np.array([clf.predict(dataset_temp)]).T))
        
        if judge == 1:
            labels = np.array([clf.predict(test_tmp)]).T
            judge = 0
        else:
            labels = np.hstack((labels,np.array([clf.predict(test_tmp)]).T))
            
labels = labels.astype(np.int64)

labels_train = labels_train.astype(np.int64)
c = 0
for i in range(labels.shape[0]):
    if np.argmax(np.bincount(labels[i])) != int(test_label_1[i]):
        #print(i,np.argmax(np.bincount(labels[i])),int(test_label_1[i]))
        c += 1
print(1-c/test_row)

c = 0
for i in range(labels_train.shape[0]):
    if np.argmax(np.bincount(labels_train[i])) != int(dataset_all_label[i]):
        #print(i,np.argmax(np.bincount(labels[i])),int(test_label_1[i]))
        c += 1
print(c/labels_train.shape[0])

0.6835443037974683
0.13291139240506328
